In [19]:
# # 安装依赖
# !pip install -U langchain
# !pip install -U langchain-deepseek
# !pip install -U langgraph
# !pip install

In [20]:
import os
from typing import TypedDict, List, Sequence, Annotated
from langchain_core.messages import (BaseMessage, HumanMessage , AIMessage, 
                                     ToolMessage, SystemMessage)
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv


In [21]:
load_dotenv(r"C:\Users\hhm18\Desktop\深度学习\env_TRPCA\apikey.env")

# This is the global variable to store document content
document_content = ""

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

@tool
def update(content: str) -> str:
    """Updates the document with the provided content."""
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is:\n{document_content}"


@tool
def save(filename: str) -> str:
    """Save the current document to a text file and finish the process.
    
    Args:
        filename: Name for the text file.
    """

    global document_content

    if not filename.endswith('.txt'):
        filename = f"{filename}.txt"


    try:
        with open(filename, 'w') as file:
            file.write(document_content)
        print(f"\n💾 Document has been saved to: {filename}")
        return f"Document has been saved successfully to '{filename}'."
    
    except Exception as e:
        return f"Error saving document: {str(e)}"
    

tools = [update, save]

llm = ChatDeepSeek(
    model="deepseek-chat",
    api_key=os.getenv("DR_API_KEY")
).bind_tools(tools)


In [22]:
def our_agent(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.
    
    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.
    
    The current document content is:{document_content}
    """)

    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)

    else:
        user_input = input("\nWhat would you like to do with the document? ")
        print(f"\n👤 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_prompt] + list(state["messages"]) + [user_message]

    response = llm.invoke(all_messages)

    print(f"\n🤖 AI: {response.content}")
    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"🔧 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

    return {"messages": list(state["messages"]) + [user_message, response]}

In [23]:
def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end the conversation."""

    messages = state["messages"]
    
    if not messages:
        return "continue"
    
    # This looks for the most recent tool message....
    for message in reversed(messages):
        # ... and checks if this is a ToolMessage resulting from save
        if (isinstance(message, ToolMessage) and 
            "saved" in message.content.lower() and
            "document" in message.content.lower()):
            return "end" # goes to the end edge which leads to the endpoint
        
    return "continue"


In [24]:
def print_messages(messages):
    """Function I made to print the messages in a more readable format"""
    if not messages:
        return
    
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\n🛠️ TOOL RESULT: {message.content}")

In [25]:
graph = StateGraph(AgentState)

graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))
graph.set_entry_point("agent")
graph.add_edge("agent", "tools")
graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue": "agent",
        "end": END,
    },
)

app = graph.compile()

In [26]:
def run_document_agent():
    print("\n ===== DRAFTER =====")
    
    state = {"messages": []}
    
    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])
    
    print("\n ===== DRAFTER FINISHED =====")

In [27]:
if __name__ == "__main__":
    run_document_agent()


 ===== DRAFTER =====

🤖 AI: Hello! I'm ready to help you create and update documents. Since you're starting fresh, I can help you draft various types of documents. 

What type of document would you like to create? For example:
- A business report or proposal
- A letter or email
- Meeting notes or minutes  
- A resume or cover letter
- A blog post or article
- Technical documentation
- Or any other type of writing

Just let me know what you'd like to work on, and I'll help you get started!

👤 USER: 续写：少年的心动像是仲夏夜的荒原，割不完烧不尽。长风一吹，野草就连了天。那悸动在胸腔里疯长，如同燎原的星火，一点就着，一燃就蔓延成无边的烈焰。

🤖 AI: 
🔧 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
少年的心动像是仲夏夜的荒原，割不完烧不尽。长风一吹，野草就连了天。那悸动在胸腔里疯长，如同燎原的星火，一点就着，一燃就蔓延成无边的烈焰。

👤 USER: 你没有续写下去？

🤖 AI: 
🔧 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
少年的心动像是仲夏夜的荒原，割不完烧不尽。长风一吹，野草就连了天。那悸动在胸腔里疯长，如同燎原的星火，一点就着，一燃就蔓延成无边的烈焰。

🛠️ TOOL RESULT: Document has been upd